In [173]:
using DataFrames
using ControlSystemsBase
using Statistics
using Distributions
using Plots
using JLD
using GLM

using RealTimeScheduling
using ControlTimingSafety

push!(LOAD_PATH, "../lib")
using Experiments
using Benchmarks

## SHT

Address the issue about equal prior belief for $H_0$ and $H_1$. Recall that given deviation upper bound $d_{ub}$, confidence level $c$, and the set of all possible samples $X$, the two hypothesis are defined as follows:
$$
\DeclareMathOperator{\dev}{Deviation}
H_0: Pr(\dev(x) < d_{ub} | x \in X) < c \\
H_1: Pr(\dev(x) < d_{ub} | x \in X) > c
$$
assuming that the sample $x$ is drawn uniformly from the set $X$.



In the paper, the Bayes Factor $B$ is used to compute $K$, the number of required samples to draw to give 
\begin{align}
\frac{Pr(H_1)}{Pr(H_0)} B &= \frac{Pr(H_1|Data)}{Pr(H_0|Data)} \\
\frac{Pr(H_1)}{Pr(H_0)} B &= \frac{\frac{Pr(Data|H_1)Pr(H_1)}{Pr(Data)}}{\frac{Pr(Data|H_0)Pr(H_0)}{Pr(Data)}} \\
B &= \frac{Pr(Data|H_1)}{Pr(Data|H_0)} \\
B &= \frac{\int_c^1 P(Data|\theta=q, H_1) f_{\theta|H_1}(q)dq}
          {\int_0^c P(Data|\theta=q, H_0) f_{\theta|H_0}(q)dq} \\
B &= \frac{\int_c^1 P(Data|\theta=q, H_1) \frac{1}{1-c} dq}
          {\int_0^c P(Data|\theta=q, H_0) \frac{1}{c} dq} \\
B &= \frac{c \int_c^1 q^K dq}
          {(1-c) \int_0^c q^K dq} \\
\frac{1-c}{c} B &= \frac{\int_c^1 q^K dq}
                        {\int_0^c q^K dq} \\
K &= -\log_c(\frac{1-c}{c}B + 1)
\end{align}

<!-- Traditionally, a Bayes factor $B$ of $100$ is considered extreme evidence favoring the alternative hypothesis. However, the paper we are based on states the type-I error is expressed by
$$
err = \frac{c}{c(1-c)B}
$$
Which is ~$.497$ when $B=100$, and ~$.000238$ when $B=4.15e5$. -->

In [3]:
let
    c = 0.99
    B = 100
    e = c/(c+(1-c)*B)
    B_og = 4.15e5
    e_og = c/(c+(1-c)*B_og)
    [e e_og]
end

1×2 Matrix{Float64}:
 0.497487  0.000238497

In [4]:
let
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    [-log.(c, c./ (1 .- c)) -log.(c, 4.15e5+1)]
end

7×2 Matrix{Float64}:
    20.8543        122.779
   457.211        1287.12
  6903.3         12929.6
 92097.8             1.29354e5
     2.30258e11      1.2936e11
     3.45672e16      1.29464e16
     3.30896e17      1.16517e17

In [5]:
let
    B = 4.15e5
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    # effective_B = c ./ (1 .- c) * B
    effective_B = (1 .- c) ./ c * B
    K = -log.(c, effective_B .+ 1)
    K_og = -log.(c, B+1)

    round.([c K_og K K_og./K], sigdigits=4)
end

7×4 Matrix{Float64}:
 0.9        122.8          101.9       1.205
 0.99      1287.0          829.9       1.551
 0.999    12930.0         6029.0       2.145
 0.9999  129400.0        37490.0       3.45
 1.0          1.294e11  415000.0  311700.0
 1.0          1.295e16  415000.0       3.12e10
 1.0          1.165e17  415000.0       2.808e11

In [6]:
let
    B = 100
    c = [0.9, 0.99, 0.999, 0.9999, 1-1e-10, 1-1e-15, 1-1e-16]
    # effective_B = c ./ (1 .- c) * B
    effective_B = (1 .- c) ./ c * B
    K = -log.(c, effective_B .+ 1)
    K_og = -log.(c, B+1)

    round.([c K_og K], sigdigits=4)
end

7×3 Matrix{Float64}:
 0.9        43.8        23.67
 0.99      459.2        69.47
 0.999    4613.0        95.35
 0.9999  46150.0        99.51
 1.0         4.615e10  100.0
 1.0         4.619e15  100.0
 1.0         4.157e16  100.0

## Experiments

### pWCET Sampler

In [7]:
sp = SamplerPWCET(0.9, 100)

SamplerPWCET(0.9, 100)

In [8]:
mean([sum(rand(sp)) for i in 1:100])

90.31

### System Dynamics

Open-loop stable system:
$$
\mathbf{\dot{x}}(t) = \begin{bmatrix} -6.0 & 1.0 \\ 0.2 & -0.7 \end{bmatrix} \mathbf{x}(t) +
\begin{bmatrix} 5.0 \\ 0.5 \end{bmatrix} u(t)
$$
where
$$
\mathbf{x}(t) = \begin{bmatrix} x_1(t) \\ x_2(t) \end{bmatrix}
$$
is the system state and $u(t)$ is the control input.

We also have a open-loop unstable system:
$$
\mathbf{\dot{x}}(t) = \begin{bmatrix} 0 & 6.5 \\ 0 & 0 \end{bmatrix} \mathbf{x}(t) +
\begin{bmatrix} 0 \\ 19.685 \end{bmatrix} u(t)
$$

In [9]:
benchmarks[:RCN], benchmarks[:F1T]

(StateSpace{ControlSystemsBase.Continuous, Float64}
A = 
 -6.0   1.0
  0.2  -0.7
B = 
 5.0
 0.5
C = 
 1.0  0.0
 0.0  1.0
D = 
 0.0
 0.0

Continuous-time state-space model, StateSpace{ControlSystemsBase.Continuous, Float64}
A = 
 0.0  6.5
 0.0  0.0
B = 
  0.0
 19.68503937007874
C = 
 1.0  0.0
D = 
 0.0

Continuous-time state-space model)

In [10]:
# Setting parameters for the experiment
H = 100
c = 0.99
B = 4.15e5
h = 0.02
sys = benchmarks[:F1T]
x0 = 1.
u0 = 0.
z0 = [fill(x0, size(sys.A, 1)); u0]

# Construct an automaton with no constraint
a = hold_kill(c2d(sys, h), delay_lqr(sys, h))

Automaton(AbstractMatrix{Float64}[[1.0 0.12999999999999998 0.02559055118110236; 0.0 1.0 0.39370078740157477; -0.582977923541146 -0.9271753376618662 -0.3501109341069651], [1.0 0.12999999999999998 0.02559055118110236; 0.0 1.0 0.39370078740157477; 0.0 0.0 1.0]], Union{Missing, Int64}[1 1], Union{Missing, Int64}[1 2], 1, [1.0 0.0 0.0])

### Simulations

In [11]:
n = 100000
res99 = sim(a, z0, 0.99, n)
res90 = sim(a, z0, 0.90, n)
res50 = sim(a, z0, 0.50, n)

100000-element Vector{Tuple{BitVector, Float64}}:
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 1, 0, 1, 1, 0, 0, 0, 1], 0.0019491733954738418)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 1, 1, 1, 1, 1, 0, 0, 1, 0], 0.002513510364079645)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 1, 1, 1, 1, 0, 0, 1, 0], 0.002722228324649051)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 0, 1, 0, 1, 1, 0, 0, 0, 1], 0.0028051305506098323)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 0, 1, 0, 1, 1, 1, 1, 0, 1], 0.0029412732882160854)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 0  …  0, 1, 1, 0, 1, 0, 0, 0, 1, 1], 0.0032532663865622)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 0, 1, 1, 1, 1, 1, 0, 1], 0.0036528180348898123)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 0  …  1, 0, 0, 1, 0, 1, 1, 1, 1, 0], 0.003725916900491611)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 0  …  1, 1, 1, 0, 1, 0, 0, 1, 1, 1], 0.0037619126506301275)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 0, 1, 0, 1, 1, 0, 0, 0, 0], 0.0038289697393772944)
 ⋮
 ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0,

In [18]:
n = 100000
res90 = sim(a, z0, 0.90, n)

100000-element Vector{Tuple{BitVector, Float64}}:
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 0.0)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 0, 1, 1], 6.216061459627196e-9)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 0, 1, 1], 6.216061459627196e-9)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 0, 1, 1, 1, 1, 1], 1.7237182695208905e-8)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 0, 1, 1, 1, 1, 1, 0, 1], 2.5251273751516622e-8)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3.847623906775773e-8)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  0, 1, 1, 1, 0, 1, 1, 1, 1, 1], 4.522184606839493e-8)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 0, 1, 1, 1, 0, 1], 4.8615861770559586e-8)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 0, 1, 1, 1, 1, 1], 5.5773123177057165e-8)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 0, 1, 1, 1, 1, 1, 1, 1], 7.179427491489854e-8)
 ⋮
 ([0, 0, 0, 0, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 0, 0, 1, 1, 0, 1, 1], 0.7

In [12]:
res10 = sim(a, z0, 0.10, n)

100000-element Vector{Tuple{BitVector, Float64}}:
 ([1, 1, 0, 1, 0, 1, 1, 0, 1, 0  …  1, 0, 0, 0, 0, 1, 1, 0, 0, 0], 0.060092451721560504)
 ([1, 0, 1, 0, 1, 1, 1, 0, 0, 0  …  1, 0, 0, 0, 0, 0, 0, 0, 0, 1], 0.09846384160314055)
 ([1, 0, 1, 0, 1, 0, 0, 1, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.10234415113983854)
 ([1, 0, 1, 0, 1, 1, 0, 0, 0, 1  …  0, 0, 0, 0, 1, 0, 0, 0, 0, 0], 0.10514906249121664)
 ([1, 0, 1, 0, 0, 1, 1, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 1, 0], 0.12713866559916698)
 ([1, 0, 1, 0, 0, 1, 0, 1, 0, 0  …  0, 0, 1, 0, 1, 0, 0, 1, 0, 0], 0.12807069602318527)
 ([1, 0, 1, 0, 1, 0, 0, 1, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.12887269580071678)
 ([0, 1, 0, 0, 1, 0, 0, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 0.13042136440054303)
 ([0, 1, 0, 0, 1, 0, 1, 0, 0, 0  …  0, 0, 0, 0, 1, 0, 0, 1, 0, 1], 0.13042136440054303)
 ([0, 1, 0, 0, 1, 0, 1, 0, 0, 0  …  0, 0, 0, 0, 1, 0, 0, 0, 1, 1], 0.13042136440054303)
 ⋮
 ([1, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0],

In [ ]:
save("../data/res99.jld", "res99", res99)
save("../data/res99.jld", "res90", res90)
save("../data/res50.jld", "res50", res50)

In [ ]:
save("../data/res50.jld", "res10", res10)

In [ ]:
devs99 = map(x -> x[2], res99)
sigm99 = map(x -> x[1], res99);

In [ ]:
map(filter(x -> misstotal(x[1]) < 3, res99)) do x
    (missfirst(x[1]), x[2])
end

In [ ]:
map(collect(filter(x -> misstotal(x[1]) < 3, res99))[end-3:end]) do x
    println(x[1], x[2])
end

In [ ]:
res = res90
devs = map(x -> x[2], res)
σs = map(x -> x[1], res)
σs3 = map(x -> x[1:3], σs)
σs5 = map(x -> x[1:5], σs)
σs7 = map(x -> x[1:7], σs)
σsx = map(x -> x[1:10], σs)
σx = σs7
p1 = plot(devs, (1:length(res)) ./ length(res), 
    xlabel="Deviation", ylabel="Probability", label="cdf", title="p=0.10")
p2 = plot(missrow.(σx), devs,
    ylabel="Deviation", xlabel="Consecutive Misses", label="p=0.9")
p3 = plot(misstotal.(σx), devs,
    ylabel="Deviation", xlabel="Total Misses")
p4 = plot(missfirst.(σs), devs,
    ylabel="Deviation", xlabel="First Miss")
for c in [0.99]
    dev = devs[round(Int64, length(res)*c)]
    scatter!(p1, [dev], [c], label="99th quantile = $(round(dev, sigdigits=3))")
end
println(length(res))
savefig(p2, "cv_cm_7.pdf")
savefig(p3, "cv_tm_7.pdf")
display(p3)

In [ ]:
for res in [res99, res90, res50, res10]
    devs = map(x -> x[2], res)
    dev = devs[round(Int64, length(res)*0.99)]
    println(round.([dev, devs[end], dev/devs[end]], sigdigits=3))
end

### Deviation Estimation

In [10]:
for p in [0.1, 0.5, 0.9, 0.99]
    mind = Inf
    maxd = 0.
    sample_t = 0
    devest_t = 0
    for i in 1:50
        t = time()
        sp = SamplerPWCET(p, 100)
        sample_t += time() - t

        t = time()
        dub = estimate_deviation(a, sp, z0, c, B)
        devest_t += time() - t

        mind = min(mind, dub)
        maxd = max(maxd, dub)
    end
    @info p sample_t devest_t mind maxd
end

┌ Info: 0.1
│   sample_t = 3.218650817871094e-5
│   devest_t = 28.934805393218994
│   mind = 63729.60023799339
│   maxd = 151435.40139364428
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:18
┌ Info: 0.5
│   sample_t = 2.1219253540039062e-5
│   devest_t = 25.836893558502197
│   mind = 2.7518499935486083
│   maxd = 11.110563807188523
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:18


┌ Info: 0.9
│   sample_t = 2.4080276489257812e-5
│   devest_t = 21.207319736480713
│   mind = 0.36408893924524055
│   maxd = 0.8886085544969886
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:18
┌ Info: 0.99
│   sample_t = 2.1219253540039062e-5
│   devest_t = 16.67701029777527
│   mind = 0.1786396896500381
│   maxd = 0.36408893924524055
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:18


In [ ]:
for p in [0.1, 0.5, 0.9, 0.99]
    mind = Inf
    maxd = 0.
    sample_t = 0
    devest_t = 0
    for i in 1:50
        t = time()
        sp = SamplerPWCET(p, 100)
        sample_t += time() - t

        t = time()
        dub = estimate_deviation(a, sp, z0, c, B)
        devest_t += time() - t

        mind = min(mind, dub)
        maxd = max(maxd, dub)
    end
    @info p sample_t devest_t mind maxd maxd/mind
end

In [ ]:
mind = Inf
maxd = 0.
x0 = 1.
u0 = 0.
for i in 1:50
    sp = SamplerPWCET(0.4, 100)
    # Construct an automaton with no constraint
    a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
    dub = estimate_deviation(a, sp, [fill(x0, size(sys.A, 1)); u0], c, B)
    mind = min(mind, dub)
    maxd = max(maxd, dub)
end
mind, maxd

In [ ]:
mind = Inf
maxd = 0.
sys = benchmarks[:F1T]
x0 = 1.
u0 = 0.
for i in 1:50
    sp = SamplerPWCET(0.9, 100)
    # Construct an automaton with no constraint
    a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
    dub = estimate_deviation(a, sp, [fill(x0, size(sys.A, 1)); u0], c, B)
    mind = min(mind, dub)
    maxd = max(maxd, dub)
end
mind, maxd

In [ ]:
mind = Inf
maxd = 0.
sys = benchmarks[:F1T]
x0 = 1.
u0 = 0.
for i in 1:50
    sp = SamplerPWCET(0.99, 100)
    # Construct an automaton with no constraint
    a = hold_kill(c2d(sys, h), delay_lqr(sys, h))
    dub = estimate_deviation(a, sp, [fill(x0, size(sys.A, 1)); u0], c, B)
    mind = min(mind, dub)
    maxd = max(maxd, dub)
end
mind, maxd

### Likelihood Ratio Test

\begin{align}
\lambda_{LR} &=\frac{\sup_{\theta_0\in\Theta_0}{L}(\theta_0)}{\sup_{\theta\in\Theta}L(\theta)} \\
&= \frac{\binom{n}{x} \theta_0^x (1-\theta_0)^{n-x}}{\binom{n}{x} \theta^x (1-\theta)^{n-x}} \\
&= \frac{\theta_0^x (1-\theta_0)^{n-x}}{ \theta^x (1-\theta)^{n-x}} \\
&= \left(\frac{\theta_0}{\theta}\right)^x + \left(\frac{1-\theta_0}{1-\theta}\right)^{n-x}
\end{align}

\begin{align}
&= -2 \ln \left[ \frac{\binom{n}{k} \theta_0^x (1-\theta_0)^{n-x}}{\binom{n}{k} \theta_1^x (1-\theta_1)^{n-x}} \right] \\
&= -2 \ln \left[ \frac{\theta_0^x (1-\theta_0)^{n-x}}{ \theta_1^x (1-\theta_1)^{n-x}} \right] \\
&= -2 (x \ln \frac{\theta_0}{\theta_1} + (n-x) \ln \frac{1-\theta_0}{1-\theta_1})
\end{align}

In [ ]:
lr_test(0.99, 100, 99)

In [ ]:
lr_test(0.99, 830, 830)

In [ ]:
lr_test_2(0.99, 8782, 0.005)

In [ ]:
lr_test_2(0.99, 46500, 0.002)

Likelihood ratio with null hypothesis: true $\theta$ is within a range (e.g., $[0.899, 0.991]$) instead of $\theta>0.99$ vs $\theta<0.99$.

In [ ]:
err = 0.000238497

In [ ]:
n = 8782
for p in [0.99, 0.90, 0.50, 0.10]
    mind = Inf
    maxd = 0.
    devest_t = 0
    for i in 1:50
        t = time()
        lrres = sim(σ p, n)
        x = round(Int64, n * 0.99)
        dev = lrres[x][2]
        devest_t += time() - t
        
        mind = min(mind, dev)
        maxd = max(maxd, dev)
    end
    @info p devest_t mind maxd maxd/mind
end

### Control Variate

In [ ]:
# Using values from simulation

res = res90
devs = map(x -> x[2], res)
σs = map(x -> x[1], res)
σs5 = map(x -> x[1:10], σs)

X = devs
Y = missrow.(σs)
Y5 = missrow.(σs5)
Z = misstotal.(σs)
Z5 = misstotal.(σs5)
U = missfirst.(σs)

data = DataFrame(Dev=X, CM=Y, TM=Z, FM=U, CM5=Y5, TM5=Z5)

ols_y = lm(@formula(Dev ~ CM5), data)

In [ ]:
ols_z = lm(@formula(Dev ~ TM5), data)

In [ ]:
ols_u = lm(@formula(Dev ~ FM), data)

In [ ]:
FcvW(0.1566, 10, res90, misstotal)

In [21]:
x = inverse_fcv(0.99, 10, res90, misstotal)

0.33958189067707595

In [24]:
res90[99000][2]

0.33958189067707595

In [91]:
confidence_interval(0.1, x, 0.01, 0.99, 9.997078246122365, res90, missfirst)

3-element Vector{Float64}:
 0.03825862867103018
 0.07624068080786475
 0.03798205213683457

In [19]:
Psi2_cv(x, 0.99, res50, misstotal)

0.004974953778107482

In [86]:
calculate_mean_miss(missfirst,res90)

9.95329

In [87]:
(100*0.9^101-101*0.9^100+1)/0.1

9.997078246122365

In [114]:
x=[1,2,3]
y=x[1:2]

2-element Vector{Int64}:
 1
 2

In [136]:
res = sim(a, z0, 0.9, 1000)
res2 = Vector{Tuple{BitVector, Float64}}(undef, 1000)
i = 1
for(σ, deviation) in res
    σs5 =  σ[1:10]
    res2[i] = (σs5, deviation)
    i += 1
end
i = 1
res2

1000-element Vector{Tuple{BitVector, Float64}}:
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.321294760989505e-6)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 2.021832394208139e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 2.3544416229805675e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 3.706448421820923e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.306530571441756e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.306530571441756e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 4.306530571441756e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 5.801510768620312e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 5.801510768620312e-5)
 ([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 5.801510768620312e-5)
 ⋮
 ([0, 0, 1, 0, 1, 1, 1, 1, 1, 1], 0.31641176846173275)
 ([0, 0, 1, 0, 1, 1, 1, 1, 1, 1], 0.31641176846173275)
 ([0, 0, 1, 1, 0, 1, 1, 1, 1, 1], 0.34104067721635434)
 ([0, 0, 1, 1, 1, 0, 0, 0, 1, 1], 0.35691285380945076)
 ([0, 0, 1, 1, 1, 1, 1, 1, 0, 1], 0.36408893924524055)
 ([0, 0, 1, 1, 1, 1, 1, 0, 1, 1], 0.36408893924524055)
 ([0, 0, 1, 1, 1, 1, 0, 1, 0, 1], 0.36408893924524055)
 ([

In [137]:
dev_CV = inverse_fcv(0.99, 1, res2, misstotal)
dev_NMC = res[990][2]
print(dev_NMC,dev_CV)

0.24909076298813280.2534311001521414

In [160]:
result = Vector{BitVector}(undef, 0)
    previous = [[1],[0]]
    for vector in previous
        push!(vector, 1)
        push!(result, vector)
    end
result

2-element Vector{BitVector}:
 [1, 1]
 [0, 1]

In [175]:
bin_list(10)

1024-element Vector{BitVector}:
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 1]
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0]
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 1]
 [1, 1, 1, 1, 1, 1, 1, 0, 1, 0]
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 1]
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 1]
 [1, 1, 1, 1, 1, 1, 0, 1, 1, 0]
 ⋮
 [0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0]
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 1]
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [178]:
n = bin_list(10)
d = [0,0,0,0,0,0,0,0,0,0]
total = 0
i = 1
for vector in n
    k = missrow(vector)
    if k != 0
        d[k] += 1
    end
end
for element in d
    total += i*element
    i+=1
end
final = total/1024

2.798828125

In [181]:
p = 0.9
mind_NMC = Inf
maxd_NMC = 0
mind_CV = Inf
maxd_CV = 0
for i in 1:50
    n = 10000
    res = sim(a, z0, p, n)
    res2 = Vector{Tuple{BitVector, Float64}}(undef, n)
    k = 1
    for(σ, deviation) in res
        σs5 =  σ[1:10]
        res2[k] = (σs5, deviation)
        k += 1
    end
    k = 1

    dev_CV = inverse_fcv(0.99, 2.798828125, res2, missrow)

    dev_NMC = res[Int(n*0.99)][2]

    mind_CV = min(mind_CV, dev_CV)
    maxd_CV = max(maxd_CV, dev_CV)
    mind_NMC = min(mind_NMC, dev_NMC)
    maxd_NMC = max(maxd_NMC, dev_NMC)

end
@info p mind_CV maxd_CV mind_NMC maxd_NMC

┌ Info: 0.9
│   mind_CV = 0.36408893924524055
│   maxd_CV = 0.5802346517806395
│   mind_NMC = 0.1838054388818805
│   maxd_NMC = 0.36408893924524055
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:28


In [139]:
p = 0.9
mind_NMC = Inf
maxd_NMC = 0
mind_CV = Inf
maxd_CV = 0
for i in 1:50
    n = 10000
    res = sim(a, z0, p, n)
    res2 = Vector{Tuple{BitVector, Float64}}(undef, n)
    k = 1
    for(σ, deviation) in res
        σs5 =  σ[1:10]
        res2[k] = (σs5, deviation)
        k += 1
    end
    k = 1

    dev_CV = inverse_fcv(0.99, (10*0.9^11-11*0.9^10+1)/0.1, res2, missfirst)

    dev_NMC = res[Int(n*0.99)][2]

    mind_CV = min(mind_CV, dev_CV)
    maxd_CV = max(maxd_CV, dev_CV)
    mind_NMC = min(mind_NMC, dev_NMC)
    maxd_NMC = max(maxd_NMC, dev_NMC)

end
@info p mind_CV maxd_CV mind_NMC maxd_NMC

┌ Info: 0.9
│   mind_CV = 0.35691285380945076
│   maxd_CV = 0.36408893924524055
│   mind_NMC = 0.18161427337879868
│   maxd_NMC = 0.36408893924524055
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:28


In [138]:
p = 0.9
mind_NMC = Inf
maxd_NMC = 0
mind_CV = Inf
maxd_CV = 0
for i in 1:50
    n = 10000
    res = sim(a, z0, p, n)
    res2 = Vector{Tuple{BitVector, Float64}}(undef, n)
    k = 1
    for(σ, deviation) in res
        σs5 =  σ[1:10]
        res2[k] = (σs5, deviation)
        k += 1
    end
    k = 1

    dev_CV = inverse_fcv(0.99, 1, res2, misstotal)

    dev_NMC = res[Int(n*0.99)][2]

    mind_CV = min(mind_CV, dev_CV)
    maxd_CV = max(maxd_CV, dev_CV)
    mind_NMC = min(mind_NMC, dev_NMC)
    maxd_NMC = max(maxd_NMC, dev_NMC)

end
@info p mind_CV maxd_CV mind_NMC maxd_NMC

┌ Info: 0.9
│   mind_CV = 0.1786396896500381
│   maxd_CV = 0.35691285380945076
│   mind_NMC = 0.1786396896500381
│   maxd_NMC = 0.35691285380945076
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:28


ErrorException: syntax: unexpected "end"

In [ ]:
p = 0.9
mind_NMC = Inf
maxd_NMC = 0
mind_CV = Inf
maxd_CV = 0
for i in 1:50
    n = 10000
    res = sim(a, z0, p, n)

    dev_CV = inverse_fcv(0.99, (100*0.9^101-101*0.9^100+1)/0.1, res, missfirst)

    dev_NMC = res[Int(n*0.99)][2]

    mind_CV = min(mind_CV, dev_CV)
    maxd_CV = max(maxd_CV, dev_CV)
    mind_NMC = min(mind_NMC, dev_NMC)
    maxd_NMC = max(maxd_NMC, dev_NMC)

end
@info p mind_CV maxd_CV mind_NMC maxd_NMC

In [129]:
p = 0.9
mind_NMC = Inf
maxd_NMC = 0
mind_CV = Inf
maxd_CV = 0
for i in 1:50
    n = 10000
    res = sim(a, z0, p, n)

    dev_CV = inverse_fcv(0.99, (100*0.9^101-101*0.9^100+1)/0.1, res, missfirst)

    dev_NMC = res[Int(n*0.99)][2]

    mind_CV = min(mind_CV, dev_CV)
    maxd_CV = max(maxd_CV, dev_CV)
    mind_NMC = min(mind_NMC, dev_NMC)
    maxd_NMC = max(maxd_NMC, dev_NMC)

end
@info p mind_CV maxd_CV mind_NMC maxd_NMC

┌ Info: 0.9
│   mind_CV = 0.1873103847440384
│   maxd_CV = 0.35691285380945076
│   mind_NMC = 0.1873103847440384
│   maxd_NMC = 0.35691285380945076
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:20


ErrorException: syntax: unexpected "end"

In [27]:
for p in [0.1, 0.5, 0.9, 0.99]
    mind_NMC = Inf
    maxd_NMC = 0
    mind_CV = Inf
    maxd_CV = 0
    for i in 1:50
        n = 10000
        res = sim(a, z0, p, n)

        dev_CV = inverse_fcv(0.99, 100*(1-p), res, misstotal)

        dev_NMC = res[Int(n*0.99)][2]

        mind_CV = min(mind_CV, dev_CV)
        maxd_CV = max(maxd_CV, dev_CV)
        mind_NMC = min(mind_NMC, dev_NMC)
        maxd_NMC = max(maxd_NMC, dev_NMC)

    end
    @info p mind_CV maxd_CV mind_NMC maxd_NMC
end

┌ Info: 0.1
│   mind_CV = 31331.40469995471
│   maxd_CV = 40661.36836285794
│   mind_NMC = 31331.40469995471
│   maxd_NMC = 40624.7811061699
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:20
┌ Info: 0.5
│   mind_CV = 1.53764165422606
│   maxd_CV = 1.7180189916789617
│   mind_NMC = 1.53764165422606
│   maxd_NMC = 1.7180189916789617
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:20


┌ Info: 0.9
│   mind_CV = 0.1873103847440384
│   maxd_CV = 0.35691285380945076
│   mind_NMC = 0.1873103847440384
│   maxd_NMC = 0.35691285380945076
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:20
┌ Info: 0.99
│   mind_CV = 0.05357866530061128
│   maxd_CV = 0.1786396896500381
│   mind_NMC = 0.05357866530061128
│   maxd_NMC = 0.1786396896500381
└ @ Main /Users/tingan/pwcet-safety/src/main.ipynb:20


## Analytic solution

### Mean of first miss
The probability $P(n)$ of first miss occurs in position $n$ in a string of length $k (k \ge n)$ with hit probablity $a (0\le a\le1)$ is:
$$
P(n)=a^{n-1}\cdot (1-a)
$$
The mean value of the position of first miss $M(k)$ can be expressed as:
$$
\begin{align}
M(k)&=\sum^{k}_{n=1}nP(n)\\
&=(1-a)\sum^{k}_{n=1}na^{n-1}\\
&=(1-a)\sum^{k}_{n=1}\frac{d}{da}a^n\\
&=(1-a)\frac{d}{da}\sum^{k}_{n=1}a^n\\
&=(1-a)\frac{d}{da}\frac{a(1-a^k)}{1-a}\\
&=\frac{ka^{k+1}-(k+1)a^k+1}{1-a}
\end{align}
$$

### Mean of total misses
The probability $P(n,k)$ of $n$ total misses occurs  in a string of length $k (k \ge n)$ with hit probablity $a (0\le a\le1)$ is:
$$
P(n,k)= {k \choose n}a^{k-n}(1-a)^n
$$
Let $1-a=x$.
The mean value of the total number of misses $M(k)$ can be expressed as:
$$
\begin{align}
M(k)&=\sum^{k}_{n=1}nP(n,k)\\
&=\sum^{k}_{n=1}n{k \choose n}a^{k-n}x^n\\
&=x\sum^{k}_{n=1}\frac{d}{dx}{k \choose n}a^{k-n}x^n\\
&=x\frac{d}{dx}(a+x)^k\\
&=k(1-a)
\end{align}

### Mean of consecutive misses
The probability $P(n,k)$ of $n$ consecutive misses occurs  in a string of length $k (k \ge n)$ with hit probablity $a (0\le a\le1)$ is:
$$
P(n,k)= 
$$